In [13]:
import csv
import datetime
import os
import pickle
import subprocess
# Imports above are standard library
# Imports below are 3rd-party
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import RocCurveDisplay, ConfusionMatrixDisplay, recall_score, precision_score, silhouette_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB, ComplementNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from statsmodels.api import qqplot
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.outliers_influence import variance_inflation_factor
import scipy.stats as stats
import statsmodels.api as sm
from xgboost import XGBClassifier, plot_importance

## Kaggle API
You will need your own API key, which you can get by following instructions at https://github.com/Kaggle/kaggle-api/blob/main/docs/README.md

In [3]:
os.environ["KAGGLE_USERNAME"] = "jsf80238"
os.environ["KAGGLE_KEY"] = "4036359324650e1c13adfc7ba87ff90e"
import kaggle  # The import itself uses your KAGGLE_KEY, that's why the import is not at the top. Really.

dataset_name = "NCHS - Death Rates and Causes of Death"
command = "kaggle datasets list --csv --search".split()
#command.append("NCHS - Death Rates and Causes of Death")
result = subprocess.run(command + [dataset_name], capture_output=True)
data = result.stdout.decode(encoding="utf-8")
data[:100]

'ref,title,size,lastUpdated,downloadCount,voteCount,usabilityRating\r\ncdc/nchs-death-rates-and-causes-'

In [4]:
csvreader = csv.DictReader(data.splitlines())
for row in csvreader:
    if row["title"] == dataset_name:
        print(row)
        dataset_ref = row['ref']
        break
dataset_ref

{'ref': 'cdc/nchs-death-rates-and-causes-of-death', 'title': 'NCHS - Death Rates and Causes of Death', 'size': '3MB', 'lastUpdated': '2019-12-28 01:24:17', 'downloadCount': '1540', 'voteCount': '14', 'usabilityRating': '0.7647059'}


'cdc/nchs-death-rates-and-causes-of-death'

In [5]:
dataset_ref = "cdc/nchs-death-rates-and-causes-of-death"
file_name = "nchs-leading-causes-of-death-united-states.csv"
command = f"kaggle datasets download --unzip --force --file {file_name} {dataset_ref}".split()
result = subprocess.run(command, capture_output=True)
stdout = result.stdout.decode(encoding="utf-8")
stderr = result.stderr.decode(encoding="utf-8")
print(stdout)
print(stderr)
print(os.linesep.join(sorted(os.listdir("."))))



100%|██████████| 834k/834k [00:00<00:00, 3.18MB/s]

.ipynb_checkpoints
death_rate.ipynb
nchs-age-adjusted-death-rates-for-selected-major-causes-of-death.csv
nchs-leading-causes-of-death-united-states.csv
socrata_metadata_nchs-age-adjusted-death-rates-for-selected-major-causes-of-death.json
socrata_metadata_nchs-death-rates-and-life-expectancy-at-birth.json
socrata_metadata_nchs-leading-causes-of-death-united-states.json
socrata_metadata_nchs-potentially-excess-deaths-from-the-five-leading-causes-of-death.json
socrata_metadata_nchs-top-five-leading-causes-of-death-united-states-1990-1950-2000.json


## Kaggle workaround
If you were unable to download the cause-of-death file you can also grab it from my GitHub repo:


In [6]:
df = pd.read_csv(file_name)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10868 entries, 0 to 10867
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year                     10868 non-null  int64  
 1   113 Cause Name           10868 non-null  object 
 2   Cause Name               10868 non-null  object 
 3   State                    10868 non-null  object 
 4   Deaths                   10868 non-null  int64  
 5   Age-adjusted Death Rate  10868 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 509.6+ KB


In [7]:
YEAR = "year"
CAUSE = "cause_name"
STATE_NAME = "state_name"
DEATH_RATE = "death_rate"
df.drop(columns=["113 Cause Name", "Deaths"], inplace=True)
name_dict = {
    "Year": YEAR,
    "Cause Name": CAUSE,
    "State": STATE_NAME,
    "Age-adjusted Death Rate": DEATH_RATE,
}
df.rename(columns=name_dict, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10868 entries, 0 to 10867
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        10868 non-null  int64  
 1   cause_name  10868 non-null  object 
 2   state_name  10868 non-null  object 
 3   death_rate  10868 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 339.8+ KB


In [8]:
mask = (df[YEAR] >= 2010) & (df[STATE_NAME] != 'United States')
df = df[mask]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4488 entries, 0 to 10802
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        4488 non-null   int64  
 1   cause_name  4488 non-null   object 
 2   state_name  4488 non-null   object 
 3   death_rate  4488 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 175.3+ KB


In [9]:
df = df[[STATE_NAME, CAUSE, DEATH_RATE]].groupby([STATE_NAME, CAUSE]).mean().reset_index()
df

,state_name,cause_name,death_rate
0,Alabama,All causes,924.6125
1,Alabama,Alzheimer's disease,35.2625
2,Alabama,CLRD,55.5750
3,Alabama,Cancer,180.5125
4,Alabama,Diabetes,22.6875
...,...,...,...
556,Wyoming,Influenza and pneumonia,17.5750
557,Wyoming,Kidney disease,11.7625
558,Wyoming,Stroke,33.2625
559,Wyoming,Suicide,24.6750


Nonemployer Statistics provides annual statistics on U.S. businesses with no paid employees or payroll at a detailed geography and industry level.

Statistics are available on businesses that have no paid employment or payroll, are subject to federal income taxes, and have receipts of $1,000 or more ($1 or more for the Construction sector). The data are available for approximately 450 NAICS industries at the national, state, county, metropolitan statistical area, and combined statistical area geography levels. The majority of NAICS industries are included.

https://www.census.gov/data/developers/data-sets/nonemp-api.2017.html#list-tab-1358655114

https://www.census.gov/data/developers/data-sets/nonemp-api.html
https://api.census.gov/data/2017/nonemp/variables.html

https://www2.census.gov/programs-surveys/nonemployer-statistics/technical-documentation/record-layouts/state-record-layout/state_record_layout_2017.txt

## Definitions
* LFO = Legal form of organization
* NAICS2017_LABEL = type of business
* NAME = geo areas, including states
* NESTAB = Number of nonemployer establishments
* NRCPTOT = Nonemployer sales, value of shipments, or revenue ($1,000)
* RCPSZES_LABEL = Separates establishments into groups by sales/receipts, including "All establishments"

In [10]:
CENSUS_API_KEY = "35b2edd7f868b6d6f79e5988091d0f8df6ffbd2a"

In [11]:
STATE_KEY = "state_key"
POPULATION = "population"
KEY = "key"
CENSUS_DEMOGRAPHICS_URL = "https://api.census.gov/data/2019/pep/charagegroups"
param_dict = {'get': 'NAME,POP', 'for': 'state:*', KEY: CENSUS_API_KEY}

In [14]:
response = requests.get(CENSUS_DEMOGRAPHICS_URL, params=param_dict)

In [16]:
result_list = response.json()  # The return from the API call is a list of lists, with first item being the column names
column_name_list = result_list.pop(0)
states_df = pd.DataFrame(result_list, columns=column_name_list)
states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NAME    52 non-null     object
 1   POP     52 non-null     object
 2   state   52 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [17]:
name_dict = {
    "NAME": STATE_NAME,
    "POP": POPULATION,
    "state": STATE_KEY,
}
states_df.rename(columns=name_dict, inplace=True)
states_df[POPULATION] = states_df[POPULATION].astype(int)
states_df[STATE_KEY] = states_df[STATE_KEY].astype(int)
states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   state_name  52 non-null     object
 1   population  52 non-null     int64 
 2   state_key   52 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ KB


In [20]:
states_df[POPULATION].sum()  # Expected value ~330 million

331433217

In [21]:
df = df.merge(states_df, on=STATE_NAME, how="inner")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   state_name  561 non-null    object 
 1   cause_name  561 non-null    object 
 2   death_rate  561 non-null    float64
 3   population  561 non-null    int64  
 4   state_key   561 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 22.0+ KB


In [22]:
NESTAB = "NESTAB"
NRCPTOT = "NRCPTOT"
LFO = "LFO"
NAICS2017 = "NAICS2017"

CENSUS_NONEMPLOYER_URL = "https://api.census.gov/data/2017/nonemp"
param_dict = {'get': ','.join((LFO, NESTAB, NRCPTOT)), 'for': 'state:*', NAICS2017: '*', KEY: CENSUS_API_KEY}

In [23]:
response = requests.get(CENSUS_NONEMPLOYER_URL, params=param_dict)

In [24]:
with open("/tmp/data", "w", newline="") as writer:
    writer.write(response.text)


In [381]:
result_list = response.json()  # The return from the API call is a list of lists, with first item being the column names
column_name_list = result_list.pop(0)
business_df = pd.DataFrame(result_list, columns=column_name_list)
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109381 entries, 0 to 109380
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   LFO        109381 non-null  object
 1   NESTAB     109381 non-null  object
 2   NRCPTOT    109381 non-null  object
 3   NAICS2017  109381 non-null  object
 4   state      109381 non-null  object
dtypes: object(5)
memory usage: 4.2+ MB


In [384]:
LEGAL_FORM_ENCODING_DICT = {
    "001": "All",
    "9101": "C Corp",
    "9111": "S Corp",
    "920": "Individual Proprietor",
    "930": "Partnership",
}
business_df["NAICS2017"].unique()

array(['00', '11', '113', '114', '1141', '11411', '1142', '11421', '115',
       '1151', '11511', '1152', '11521', '1153', '11531', '21', '211',
       '2111', '21112', '21113', '212', '2121', '21211', '2122', '2123',
       '213', '2131', '21311', '22', '221', '23', '236', '2361', '2362',
       '237', '2371', '2372', '23721', '2373', '23731', '2379', '23799',
       '238', '2381', '2382', '23821', '23822', '23829', '2383', '2389',
       '31-33', '311', '3111', '31111', '3112', '3113', '3114', '3115',
       '3116', '31161', '31171', '3117', '3119', '3118', '312', '313',
       '314', '315', '316', '3161', '31611', '3162', '31621', '3169',
       '31699', '321', '322', '323', '3231', '324', '3241', '325', '3251',
       '3252', '3253', '3254', '32541', '3256', '3255', '3259', '326',
       '327', '3271', '3272', '32721', '3273', '3274', '3279', '331',
       '332', '333', '334', '335', '336', '337', '339', '3391', '33911',
       '3399', '42', '423', '4231', '4232', '4233', '4234', '

In [349]:
LEGAL_FORM = "legal_form"
BUSINESS_TYPE = "business_type"
ESTABLISHMENT_COUNT = "establishment_count"
REVENUE_IN_THOUSANDS = "revenue_in_thousands"
REVENUE_CATEGORY = "revenue_category"
name_dict = {
    "state": STATE_KEY,
    "NAICS2017": BUSINESS_TYPE,
    "LFO": LEGAL_FORM,
    "NESTAB": ESTABLISHMENT_COUNT,
    "NRCPTOT": REVENUE_IN_THOUSANDS,
}
business_df.rename(columns=name_dict, inplace=True)
business_df = business_df.reindex([STATE_KEY, LEGAL_FORM, BUSINESS_TYPE, ESTABLISHMENT_COUNT, REVENUE_IN_THOUSANDS], axis=1)
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109381 entries, 0 to 109380
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   state_key             109381 non-null  object
 1   legal_form            109381 non-null  object
 2   business_type         109381 non-null  object
 3   establishment_count   109381 non-null  object
 4   revenue_in_thousands  109381 non-null  object
dtypes: object(5)
memory usage: 4.2+ MB


In [350]:
for column_name in ESTABLISHMENT_COUNT, REVENUE_IN_THOUSANDS, STATE_KEY:
    business_df[column_name] = business_df[column_name].astype(int)
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109381 entries, 0 to 109380
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   state_key             109381 non-null  int64 
 1   legal_form            109381 non-null  object
 2   business_type         109381 non-null  object
 3   establishment_count   109381 non-null  int64 
 4   revenue_in_thousands  109381 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 4.2+ MB


In [351]:
business_df

,state_key,legal_form,business_type,establishment_count,revenue_in_thousands
0,28,001,00,219596,8889009
1,28,9101,00,1750,206357
2,28,9111,00,6527,848178
3,28,920,00,198380,6220549
4,28,930,00,12939,1613925
...,...,...,...,...,...
109376,26,001,813,5972,86303
109377,26,9101,813,135,13364
109378,26,9111,813,19,2159
109379,26,920,813,5803,69991


In [352]:
mask = business_df[[STATE_KEY, BUSINESS_TYPE, LEGAL_FORM]].duplicated()
business_df[mask]

,state_key,legal_form,business_type,establishment_count,revenue_in_thousands


Verify this data does not have holes,
meaning verify every combination of legal_form/business_type exists for every state.
If something is missing, add it with zero for the establishment_count and revenue_in_thousands.

In [353]:
from itertools import product
state_list = business_df[STATE_KEY].unique()
business_type_list = business_df[BUSINESS_TYPE].unique()
legal_form_list = business_df[LEGAL_FORM].unique()
wanted_combinations_set = set(product(state_list, business_type_list, legal_form_list))
print(f"{len(wanted_combinations_set):,} possible combinations, for example:")
for i, item in enumerate(wanted_combinations_set):
    print(item)
    if i > 10:
        break


119,850 possible combinations, for example:
(40, '21113', '9101')
(37, '44612', '001')
(41, '21311', '9101')
(23, '5615', '930')
(5, '2381', '9101')
(56, '7114', '930')
(32, '5322', '930')
(39, '5611', '9101')
(41, '522', '9101')
(40, '221', '001')
(47, '4246', '930')
(11, '6219', '001')


In [354]:
existing_combinations_set = set()
for item in business_df[[STATE_KEY, BUSINESS_TYPE, LEGAL_FORM]].to_records(index=False):
    existing_combinations_set.add(tuple(item))
print(f"{len(existing_combinations_set):,} existing combinations, for example:")
for i, item in enumerate(existing_combinations_set):
    print(item)
    if i > 10:
        break

109,381 existing combinations, for example:
(40, '21113', '9101')
(37, '44612', '001')
(41, '21311', '9101')
(23, '5615', '930')
(5, '2381', '9101')
(56, '7114', '930')
(32, '5322', '930')
(39, '5611', '9101')
(41, '522', '9101')
(40, '221', '001')
(47, '4246', '930')
(11, '6219', '001')


In [355]:
missing_combinations_set = wanted_combinations_set - existing_combinations_set
print(f"{len(missing_combinations_set):,} missing combinations, for example:")
for i, item in enumerate(missing_combinations_set):
    print(item)
    if i > 10:
        break

10,469 missing combinations, for example:
(27, '324', '9101')
(46, '451212', '9111')
(28, '2121', '9101')
(50, '3161', '930')
(33, '7224', '9111')
(45, '2121', '930')
(46, '44531', '9101')
(46, '3114', '9101')
(2, '81221', '9101')
(31, '7213', '9101')
(15, '48541', '930')
(54, '487', '9101')


In [356]:
missing_data = [list(combo) + [0, 0] for combo in missing_combinations_set]
missing_data[:10]

[[27, '324', '9101', 0, 0],
 [46, '451212', '9111', 0, 0],
 [28, '2121', '9101', 0, 0],
 [50, '3161', '930', 0, 0],
 [33, '7224', '9111', 0, 0],
 [45, '2121', '930', 0, 0],
 [46, '44531', '9101', 0, 0],
 [46, '3114', '9101', 0, 0],
 [2, '81221', '9101', 0, 0],
 [31, '7213', '9101', 0, 0]]

In [357]:
missing_df = pd.DataFrame.from_records(missing_data)
missing_df.columns = [STATE_KEY, LEGAL_FORM, BUSINESS_TYPE, ESTABLISHMENT_COUNT, REVENUE_IN_THOUSANDS]
missing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10469 entries, 0 to 10468
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   state_key             10469 non-null  int64 
 1   legal_form            10469 non-null  object
 2   business_type         10469 non-null  object
 3   establishment_count   10469 non-null  int64 
 4   revenue_in_thousands  10469 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 409.1+ KB


In [358]:
business_df = pd.concat([business_df, missing_df], ignore_index=True)
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119850 entries, 0 to 119849
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   state_key             119850 non-null  int64 
 1   legal_form            119850 non-null  object
 2   business_type         119850 non-null  object
 3   establishment_count   119850 non-null  int64 
 4   revenue_in_thousands  119850 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 4.6+ MB


In [359]:
df = df.merge(business_df, on=STATE_KEY)
df.drop(columns=[STATE_KEY], inplace=True)
df = df.reindex([CAUSE, STATE_NAME, POPULATION, LEGAL_FORM, BUSINESS_TYPE, ESTABLISHMENT_COUNT, REVENUE_IN_THOUSANDS, DEATH_RATE], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1318350 entries, 0 to 1318349
Data columns (total 8 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   cause_name            1318350 non-null  object 
 1   state_name            1318350 non-null  object 
 2   population            1318350 non-null  int64  
 3   legal_form            1318350 non-null  object 
 4   business_type         1318350 non-null  object 
 5   establishment_count   1318350 non-null  int64  
 6   revenue_in_thousands  1318350 non-null  int64  
 7   death_rate            1318350 non-null  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 80.5+ MB


In [367]:
LEGAL_FORM, BUSINESS_TYPE

('legal_form', 'business_type')

In [364]:
frames_by_cause_dict = dict()
for cause_name in df[CAUSE].unique():
    mask = (df[CAUSE] == cause_name)
    new_df = df[mask].copy()
    new_df.drop(columns=[CAUSE], inplace=True)
    frames_by_cause_dict[cause_name] = new_df
frames_by_cause_dict["Heart disease"].info()

<class 'pandas.core.frame.DataFrame'>
Index: 119850 entries, 11750 to 1306599
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   state_name            119850 non-null  object 
 1   population            119850 non-null  int64  
 2   legal_form            119850 non-null  object 
 3   business_type         119850 non-null  object 
 4   establishment_count   119850 non-null  int64  
 5   revenue_in_thousands  119850 non-null  int64  
 6   death_rate            119850 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 7.3+ MB
